# Question A

In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
yelp = pd.read_csv("Yelp Data Restaurant Reviews Ratings.csv")

high_mask = yelp['stars'] > 3
yelp['High'] = 0
yelp.ix[high_mask, 'High'] = 1


#####logistic regression
from patsy import dmatrices

formula = 'High ~ 0 + votes_cool + votes_funny+ votes_useful + Cheap + Moderate + Expensive + VeryExpensive + American + Chinese + French + Japanese + Indian + Italian + Greek + Mediterranean + Mexican + Thai + Vietnamese + Others'

Y, X = dmatrices(formula, yelp, return_type='dataframe')
y = Y['High'].values

#Split the data into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#Fit the classifier
from sklearn.linear_model import LogisticRegression
model_logistic = LogisticRegression()
result_logistic = model_logistic.fit(X_train, y_train)

from sklearn import metrics

prediction_train_logistic = model_logistic.predict(X_train)
print 'the accurancy of logistic regression model we built in training data set is ', metrics.accuracy_score(y_train, prediction_train_logistic)
prediction_test = model_logistic.predict(X_test)
print 'the accurancy of logistic regression model we built in test data set is ', metrics.accuracy_score(y_test, prediction_test)

the accurancy of logistic regression model we built in training data set is  0.684977498393
the accurancy of logistic regression model we built in test data set is  0.685


In [3]:
##### KNN
from sklearn import neighbors
model = neighbors.KNeighborsClassifier(n_neighbors=15,
                                       weights='uniform',
                                       p=2)
model.fit(X_train, y_train)
print 'We also build a KNN model to check whether we could improve the accurancy or not, here is the result: '
print 'The accurancy of KNN model we built in training data set is' , metrics.accuracy_score(y_train, model.predict(X_train))
print 'The accurancy of KNN model we built in test data set is', metrics.accuracy_score(y_test, model.predict(X_test))
print 
print 'We could see that the KNN and the logistic regression has very similar accurancies. '

We also build a KNN model to check whether we could improve the accurancy or not, here is the result: 
The accurancy of KNN model we built in training data set is 0.687406243303
The accurancy of KNN model we built in test data set is 0.6665

We could see that the KNN and the logistic regression has very similar accurancies. 


# Question B

In [4]:
data_b = yelp[['Review','High']]

import numpy as np
np.random.seed(1)
train = data_b.sample(int(len(data_b)*0.5), replace=False)
test = data_b[~data_b.index.isin(train.index.values)]

train_x = train['Review']
train_y = train['High']
test_x = test['Review']
test_y = test['High']

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0,smooth_idf=True, strip_accents='unicode', norm='l2')

X_train=vectorizer.fit_transform(train_x)
X_test=vectorizer.transform(test_x) 

from sklearn.naive_bayes import MultinomialNB
nb_classifier1=MultinomialNB().fit(X_train, train_y)
pred = nb_classifier1.predict(X_test)
pred = Series(pred).reset_index()[0]

df=pd.DataFrame()
df['Predicted']=pred
df['Actual']=test_y.reset_index()['High']


In [5]:
print "The accurancy of the Naive Bayes model we built with only text data is",(df['Predicted']==df['Actual']).mean()
print 
print 'We could also see that the accurancy is very similar as logistic regression model and KNN model we build with only numeric variables '

The accurancy of the Naive Bayes model we built with only text data is 0.6858

We could also see that the accurancy is very similar as logistic regression model and KNN model we build with only numeric variables 


# Question C

In [6]:
np.random.seed(1)
train = yelp.sample(int(len(yelp)*0.5), replace=False)
test = yelp[~yelp.index.isin(train.index.values)]

train_x = train['Review']
train_y = train['High']
test_x = test['Review']
test_y = test['High']

X_train=vectorizer.fit_transform(train_x)
X_test=vectorizer.transform(test_x) 

import scipy.sparse
a=train.drop(["Review",'stars',"High"],axis=1).copy()
a_sparse=scipy.sparse.csr_matrix(a.to_sparse()) 
b=scipy.sparse.hstack([X_train, a_sparse])    

In [7]:
# do the same for test set
c=test.drop(["Review","stars","High"], axis=1).copy() 
c_sparse=scipy.sparse.csr_matrix(c.to_sparse()) 
d=scipy.sparse.hstack([X_test, c_sparse]) 
    
nb_classifier2 = MultinomialNB().fit(b, train_y)
pred2 = nb_classifier2.predict(d)
pred2=Series(pred2).reset_index()[0]

df=pd.DataFrame()
df['Predicted']=pred2
df['Actual']=test_y.reset_index()['High']
    

In [8]:
print "The accuracy of Naive Bayes model we build with text and numeric data is ",(df['Predicted']==df['Actual']).mean()
print 'Weird thing happend: All the models we built seems has very similar accuracy. Why is that?'
print 
print 'We find out that all our models predict almost very test data HIGH. And the percentage of the HIGH in the dataset is around 68%. So that is why our models all got around 68% accurancies. '
print 
print 'There might be something wrong with our models, but we run out of time to exam what is wrong with that'

The accuracy of Naive Bayes model we build with text and numeric data is  0.6853
Weird thing happend: All the models we built seems has very similar accuracy. Why is that?

We find out that all our models predict almost very test data HIGH. And the percentage of the HIGH in the dataset is around 68%. So that is why our models all got around 68% accurancies. 

There might be something wrong with our models, but we run out of time to exam what is wrong with that


# Question D

In [9]:
# Part D
import pandas as pd
import numpy as np
from pandas import Series
from pandas import DataFrame
from patsy import dmatrices
from sklearn.cross_validation import train_test_split,StratifiedShuffleSplit,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import neighbors
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import scipy.sparse
import sklearn.cluster
from nltk import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
import itertools
import math
raw=pd.read_csv('Yelp Data Restaurant Reviews Ratings.csv')
result=pd.read_excel('Yelpresults.xlsx')
raw['High'] = 0
raw.ix[raw['stars']>3, 'High'] = 1
result['Combo_Score']=result['Pos Senti']+result['Neg Senti']
result['Positivity']=0

In [10]:
result.ix[result['Combo_Score']>0, 'Positivity']=1
result['Target']=raw['High']
result[:10]
from sklearn import metrics

In [11]:
print 'Using sentistrength, the confusion matrix of the predicted value and the target value confusion matrix is', '\n', metrics.confusion_matrix(result['Positivity'],result['Target'])
print 'The accuracy is', round(metrics.accuracy_score(result['Positivity'],result['Target']),3)*100,"%"
print 
print 'Here we could actually choose the threshold of the Combo_Score. Here we choose 0 as our threshold. Combo_Score above 0 will be set to HIGH, and lower than 0 will set to LOW \n'
print 'We could see that the accuracy is not very good, however, we did see the improvement of this model than all the other three models in A, B, C in predicting LOW.'

Using sentistrength, the confusion matrix of the predicted value and the target value confusion matrix is 
[[2354 4212]
 [4092 9340]]
The accuracy is 58.5 %

Here we could actually choose the threshold of the Combo_Score. Here we choose 0 as our threshold. Combo_Score above 0 will be set to HIGH, and lower than 0 will set to LOW 

We could see that the accuracy is not very good, however, we did see the improvement of this model than all the other three models in A, B, C in predicting LOW.


# Question E

In [12]:
##############
import pandas as pd
rowdf = pd.read_csv('Yelp Data Restaurant Reviews Ratings.csv')
#redf = pd.read_csv('Yelp Rating+results 1', sep=', ')

#rowdf[:5]
#redf[:5]

In [13]:
###############
rowdf['High'] = 0
rowdf.ix[rowdf['stars']>3,'High'] = 1

#### Define a tokenlization function 'preprocess'

In [14]:
####################
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    #r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

#reviews2 = list()

#for review in rowdf['Review']:
#    review = review.lower()
#    a = preprocess(review)
#    reviews2.append(a)
    

#print reviews2[:5]

In [15]:
##############
from collections import Counter

from nltk.corpus import stopwords
import string
from nltk import bigrams 

count_all2=Counter()
count_all3=Counter()  # bigram

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation 


#word_frequencies = [Counter(review) for review in reviews2]
#document_frequencies = Counter()
#map(document_frequencies.update, (word_frequency.keys() for word_frequency in word_frequencies))
#print(document_frequencies)

#### Get a list of words and each word's count. By get the top counts words list, we could choose out what is the most popular positive words and negative words in this context. Then we could get a list of positive words and negative words. 

In [16]:
##################
wordlist = []
for review in rowdf['Review']:
        review = review.lower()
        # Create a list with all the terms
        terms_stop = [term for term in preprocess(review) if term not in stop]
        wordlist.append(terms_stop)
        # Update the counter
        count_all2.update(terms_stop)
        terms_bigram = bigrams(terms_stop)
        count_all3.update(terms_bigram)
    # Print the first 5 most frequent words
print(count_all2.most_common(10))
print(count_all3.most_common(10))
print wordlist[:2]

C:\Users\Summer\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


[('food', 17499), ('good', 16147), ('place', 14277), ('great', 10796), ('like', 10347), ('one', 8105), ('service', 7638), ('really', 7391), ('go', 7321), ('get', 7085)]
[(('happy', 'hour'), 1569), (('go', 'back'), 1462), (('mexican', 'food'), 1188), (('really', 'good'), 1036), (('pretty', 'good'), 967), (('food', 'good'), 909), (('first', 'time'), 833), (('good', 'food'), 764), (('next', 'time'), 741), (('chips', 'salsa'), 708)]
[['location', 'business', 'drove', 'way', 'costco', 'giant', 'lease', 'sign'], ['closed', 'jb', 'location', 'poor', 'building', 'layout', 'poor', 'service', 'mediocre', 'food', 'finally', 'folded', 'good', 'riddance', ':-)']]


#### Change all the positive words into 'positive'. Change all the negative words into 'negative'. In this case, it is faster to run the code

In [17]:
positive_vocab = ['good', 'nice', 'great', 'awesome', 'outstanding', 'wonderful', 'love','like','yummy',':)',
                  ':-)','best','delicious','fresh','friendly','tasty','amazing','perfect','loved']
negative_vocab = ['hate', 'awful', 'terrible', 'disgusting', 'gross', 'horrible', 'dislike',':(',': (',
                 'poor','worst']

def cw_pn(review): 
    for i in range(len(review)-1):
        if review[i] in positive_vocab: review[i]='positive'
        if review[i] in negative_vocab: review[i]='negative'
    return review

wordlist2=[cw_pn(review) for review in wordlist]

#### get a list of words and their count in 'count_all4'. get a litst of bigrams and their count in 'count_all5'  

In [18]:
##################
# wordlist3 = []
count_all4=Counter()
count_all5=Counter()

for review in wordlist2:
        count_all4.update(review)
        terms_bigram = bigrams(review)
        count_all5.update(terms_bigram)
    # Print the first 5 most frequent words
print(count_all4.most_common(10))
#print(count_all5.most_common(40))
print wordlist2[:2]

[('positive', 72737), ('food', 17499), ('place', 14277), ('one', 8105), ('service', 7638), ('really', 7391), ('go', 7321), ('get', 7085), ('time', 6751), ('back', 6662)]
[['location', 'business', 'drove', 'way', 'costco', 'giant', 'lease', 'sign'], ['closed', 'jb', 'location', 'negative', 'building', 'layout', 'negative', 'service', 'mediocre', 'food', 'finally', 'folded', 'positive', 'riddance', ':-)']]


#### POS each word, in order to find the bigrams in some special format

In [19]:
%pylab inline
import nltk as n
# n.download()
from nltk.tokenize import sent_tokenize, word_tokenize

wordtags=[]
for review in wordlist2: 
    token_alpha=[t for t in review if t.isalpha()]
    pos_tags=n.pos_tag(token_alpha)
    wordtags.append(pos_tags)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


#### find the bigrams in some special format, that is more informative than other bigrams. All the special bigrams stored in bigramlis

In [20]:
bigramlist=[]
count_all_bigram=Counter()


for review in wordtags:
    for i in range(len(review)-1):
        type1 = review[i-1][1]
        type2 = review[i][1]
        bi=[]
        #print type1
        #print type2
        if (((type1 == 'JJ') and (type2=='NN')) or 
            ((type1 == 'JJ') and (type2=='NNS')) or
            ((type1 == 'RB') and (type2=='JJ')) or
            ((type1 == 'RBR') and (type2=='JJ')) or
            ((type1 == 'RBS') and (type2=='JJ')) or
            ((type1 == 'JJ') and (type2=='JJ') )or
            ((type1 == 'NN') and (type2=='JJ') )or
            ((type1 == 'NNS') and (type2=='JJ') ) ):  
            bi.append((review[i-1][0],review[i][0]))
            bigramlist.append((review[i-1][0],review[i][0]))
        count_all_bigram.update(bi)
print bigramlist[:5]
print count_all_bigram.most_common(40)

[('costco', 'giant'), ('giant', 'lease'), ('riddance', 'closed'), ('closed', 'jb'), ('location', 'negative')]
[(('positive', 'positive'), 3062), (('food', 'positive'), 2659), (('positive', 'food'), 2422), (('really', 'positive'), 2212), (('positive', 'place'), 2065), (('happy', 'hour'), 1522), (('service', 'positive'), 1486), (('positive', 'service'), 1470), (('pretty', 'positive'), 1206), (('place', 'positive'), 1134), (('mexican', 'food'), 1124), (('always', 'positive'), 820), (('first', 'time'), 808), (('also', 'positive'), 737), (('next', 'time'), 637), (('staff', 'positive'), 600), (('last', 'night'), 576), (('positive', 'mexican'), 477), (('positive', 'little'), 427), (('positive', 'staff'), 412), (('atmosphere', 'positive'), 398), (('positive', 'thing'), 373), (('quite', 'positive'), 364), (('chinese', 'food'), 358), (('super', 'positive'), 350), (('absolutely', 'positive'), 343), (('still', 'positive'), 338), (('sweet', 'potato'), 333), (('positive', 'meal'), 326), (('last', 't

In [28]:
bigrams=[]
for bi in count_all_bigram.most_common(1000):
    bigrams.append(bi[0])


#### caculate the counts of co-occurrence of bigrams and other positive words and negative words within 5 word distance

In [29]:
###########
from collections import defaultdict
# remember to include the other import from the previous post
com_bigram = defaultdict(lambda : defaultdict(int))

for review in wordlist2:  
    # Build co-occurrence matrix between bigram and positive and negative
    for i in range(len(review)-1):  
        if (review[i-1],review[i]) in bigrams: 
            for j in range(max([0,i-5]), min(i+5,len(review)-1)):
                #w1, w2 = sorted([(review[i-1],review[i]), review[j]])             
                com_bigram[(review[i-1],review[i])][review[j]] += 1
                
print com_bigram[('positive', 'service')]['positive']

2698


In [30]:
n_p = 0.01 + count_all4['positive']
n_n = 0.01 + count_all4['negative']

#### caculate the semantic_orientation of each bigrams we choose. 

In [31]:
###############
import math

semantic_orientation = {}
for (word1,word2) in bigrams:
    pos = 0.01 # counter for number of cases with phrase and positive word
    neg = 0.01 # counter for number of cases with phrase and negative word
    pos += com_bigram[(word1,word2)]['positive'] 
    neg += com_bigram[(word1,word2)]['negative'] 
    numer = pos * n_n
    denom = neg * n_p
    semantic_orientation[(word1,word2)] = math.log(numer/denom) / math.log(2)
    
semantic_orientation

{('happy', 'hour'): 1.2292434249274624,
 ('food', 'excellent'): -0.3299959816661807,
 ('large', 'portions'): 7.2295084335006825,
 ('try', 'positive'): 1.0109245189727358,
 ('positive', 'lots'): 8.259725431709676,
 ('appetizer', 'positive'): 8.101045158268626,
 ('italian', 'beef'): 5.762923893970331,
 ('items', 'positive'): 8.40266920953105,
 ('last', 'visit'): -2.1660027617797097,
 ('positive', 'margarita'): 7.31875735801452,
 ('positive', 'first'): 1.5858797221754397,
 ('next', 'door'): -0.49270251666984183,
 ('surprisingly', 'positive'): 1.1040761620170938,
 ('vietnamese', 'food'): 6.3756071522499385,
 ('phoenix', 'positive'): 1.3479436879715534,
 ('little', 'bland'): 6.3756071522499385,
 ('positive', 'french'): 8.134208210023036,
 ('fantastic', 'positive'): 8.244620121056293,
 ('staff', 'attentive'): -0.02893720383222598,
 ('totally', 'worth'): 5.376161715024742,
 ('positive', 'tortilla'): 8.032340590392296,
 ('positive', 'feel'): 0.906326273375872,
 ('positive', 'sauce'): 1.8180418

#### using the semantic_orientation of each bigrams, we could caculated the score of sentiment of each review

In [54]:
test = rowdf
wordframe = DataFrame({'wordlist2': wordlist2})

def func(review):
    score = 0.01
    total = 0.01
    #print score
    for i in range(len(review)-1):
        #print i
        if (review[i],review[i+1]) in bigrams:
            score += semantic_orientation[(review[i],review[i+1])]
            total += 1    
    return score/total     

test['Avg'] = wordframe['wordlist2'].apply(func)


#### By choose the threshold, we could set HIGH and LOW to each review. 

In [55]:
test['High_P'] = 0
test.ix[rowdf['Avg']>=0.2,'High_P'] = 1

In [57]:
from sklearn import metrics
print 'the confusion matrix of is \n', metrics.confusion_matrix(test['High'], test['High_P'])
print 'the accurancyis', metrics.accuracy_score(test['High'], test['High_P'])

print 'We improved the accurancy! Yeah! '
print '\n we find there are some problems with our model which we did not solve'
print '\n first, lots of bigrams itself actually contain postive or negative words.They already have a sentiment. But we caculated an avarage sentiment for them '
print '\n second, we didn,t catch the sentiments of words like isn,t don,t doesn,t didn,t in our models, so sometimes it means a negative meaning like isn,t very good, but we treat this as very good.'
print '\n These problems could be fixed, but it took more efforts to do so.'

the confusion matrix of is 
[[  646  5800]
 [  190 13363]]
the accurancyis 0.700485024251
We improved the accurancy! Yeah! 

 we find there are some problems with our model which we did not solve

 first, lots of bigrams itself actually contain postive or negative words.They already have a sentiment. But we caculated an avarage sentiment for them 

 second, we didn,t catch the sentiments of words like isn,t don,t doesn,t didn,t in our models, so sometimes it means a negative meaning like isn,t very good, but we treat this as very good.

 These problems could be fixed, but it took more efforts to do so.


# Question F

In [35]:
# Part F
# Get high and low reviews and tokenize, lowercase
reviews = raw['Review']
reviews = reviews.str.decode('utf-8')
high=reviews[raw['High']==1].copy()
low=reviews[raw['High']==0].copy()
high=high.map(word_tokenize)
low=low.map(word_tokenize)
high=itertools.chain.from_iterable(high)
high_lower=[i.lower() for i in high if i.isalpha()]
low=itertools.chain.from_iterable(low)
low_lower=[i.lower() for i in low if i.isalpha()]

# When in doubt, value counts
high_vc=pd.Series(high_lower).value_counts()
low_vc=pd.Series(low_lower).value_counts()

# Get top 100 words to tag and only select top 5 nouns for attributes
high_bowdlerized=[i for i in high_vc.index if i not in stopwords.words('english')]
low_bowdlerized=[i for i in low_vc.index if i not in stopwords.words('english')]
high_tag100 = pos_tag(high_bowdlerized[:100])
low_tag100 = pos_tag(low_bowdlerized[:100])
high_NN = [i[0] for i in high_tag100 if i[1]=='NN']
low_pos100 = pos_tag(low_bowdlerized[:100])
low_NN = [i[0] for i in low_pos100 if i[1]=='NN']
low_top5 = low_NN[:5]
print high_NN[:5]
print low_top5
print 'Top 5 attributes for good restaurants are food (no doubt), location/ambience, queuing time, artistry of the menu.'
print 'Top 5 attributes for bad restaurants are food quality, waiting time, order, etc.'

[u'food', u'place', u'time', u'love', u'menu']
[u'food', u'place', u'get', u'time', u'order']
Top 5 attributes for good restaurants are food (no doubt), location/ambience, queuing time, artistry of the menu.
Top 5 attributes for bad restaurants are food quality, waiting time, order, etc.
